In [1]:
import networkx as nx
import math
import matplotlib.pyplot as plt
from networkx.algorithms import tree
import pandas
import pandasgen
import os
import subprocess

In [2]:
def read_edges():
    metrics = ['cell', 'col'] #, 'rel'
    edges_metrics = []
    for metric in metrics:
        file_name='/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/src/preserving_ops/infered_mst_' + metric + '.csv'
        edges = []
        with open(file_name) as f:
            for line in f:
                tokens = line.strip().split(',')
                e = (tokens[0], tokens[1], tokens[2])
                edges.append(e)
        edges_metrics.append(edges)
    return edges_metrics

In [3]:
######## Synthetic dataset generation
def data_gen(row_cnt, col_cnt, version_cnt, branch_factor, freq):
    ds,errors = pandasgen.generate__dataset((row_cnt,col_cnt), version_cnt, scale=branch_factor, snapshot_freq=freq)
    print (errors)
    # Visualize this workflow
    #ds.lineage.draw_graph(canvas_size=(50,50))
    return ds.lineage.graph.edges

In [4]:
####### Building Blocks for lineage inference
def profiling(dir_path, endwith):
    files=sorted(os.listdir(dir_path))
    print (files)
    artifacts = ''
    for f in files:
        if f.endswith(endwith): #f.startswith('repo'): #
            artifacts += dir_path + f + ','
    print ("----------profiling ----------")
    proc = subprocess.Popen(['/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/src/profiling/profiling', '-dir', artifacts], stdout=subprocess.PIPE)
    output = proc.communicate()[0]
    print('Got stdout:', str(output).replace('\\r', '\n').replace('\\n', '\n'))


def pre_clustering(clustering_strategy):
    print ("---------pre_clustering----------")
    proc = subprocess.Popen(['/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/src/pre_clustering/pre_clustering', clustering_strategy], stdout=subprocess.PIPE)
    output = proc.communicate()[0]
    print('Got stdout:', str(output).replace('\\r', '\n').replace('\\n', '\n'))

def clear_files():
    cmd = 'rm /home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/*.csv /home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/src/profiling/*.csv /home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/src/pre_clustering/*.csv /home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/src/preserving_ops/result/*.csv /home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/src/preserving_ops/*.csv'
    os.system(cmd)

def preserving_ops(dir_path, endwith):
    files=sorted(os.listdir(dir_path))
    print (files)
    artifacts = ''
    for f in files:
        if f.endswith(endwith): #f.startswith('repo'): #
            artifacts += dir_path + f + ','
    print ("----------preserving-ops----------")
    proc = subprocess.Popen(['/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/src/preserving_ops/preserving_ops', '-dir', artifacts], stdout=subprocess.PIPE)
    output = proc.communicate()[0]
    print('Got stdout:', str(output).replace('\\r', '\n').replace('\\n', '\n'))

In [5]:
def lineage_inference(pre_clustering_strategy):
    profiling('/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/', '.csv') #retail/') #nb_123977/artifacts/')
    pre_clustering(pre_clustering_strategy) #('-exact_schema')  #('-no_pre_cluster')#
    preserving_ops('/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/', '.csv') #nb_123977/artifacts/') # /repo_user/')

In [6]:
### calculate statistics
def calculate_common_edge(inferred):
    result = []
    for infer_edges in inferred:
        common = 0
        for e in infer_edges:
            key = e[0] + " " + e[1]
            if key in e_dict:
                common += 1
        result.append(common)
    return result

In [ ]:
row_cnt = 10000
col_cnt = 20
branch_factor = 1.5
version_cnt = [80] # 20, 40, 50
materialize_freq = [2] # [,2,3,4,5]
repeat = 10

# common_edges = {}
for v_cnt in version_cnt:
    for freq in materialize_freq:
        result = [] # 10 sample exp
        for i in range(repeat):
            # generate data
            clear_files()
            real_edges = data_gen(row_cnt, col_cnt, v_cnt, branch_factor, freq)
            e_dict = {}
            for e in real_edges:
                e_dict[e[0] + ".csv " + e[1] + ".csv"] = 1
                e_dict[e[1]+ ".csv " + e[0]+ ".csv"]= 1
            lineage_inference('-no_pre_cluster')
            inferred = read_edges()
            no_pre_cluster_result = calculate_common_edge(inferred)
            lineage_inference('-exact_schema')
            inferred = read_edges()
            exact_schema_result = calculate_common_edge(inferred)
            result.append([no_pre_cluster_result, exact_schema_result])
        common_edges[str(v_cnt) + ',' + str(freq)] = result

[{'drop_cols': 'Traceback (most recent call last):\n  File "/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/pandasgen.py", line 318, in generate__dataset\n    dataset.apply_ops(ops)\n  File "/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/pandasgen.py", line 85, in apply_ops\n    raise pd.errors.EmptyDataError\npandas.errors.EmptyDataError\n'}, {'drop_rows': 'Traceback (most recent call last):\n  File "/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/pandasgen.py", line 318, in generate__dataset\n    dataset.apply_ops(ops)\n  File "/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/pandasgen.py", line 85, in apply_ops\n    raise pd.errors.EmptyDataError\npandas.errors.EmptyDataError\n'}, {'iloc': 'Traceback (most recent call last):\n  File "/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/pandasgen.py", line 318, in generate__dataset\n    dataset.apply_ops(ops)\n  Fil

Got stdout: b'/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/0.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/1.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/10.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/11.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/12.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/13.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/14.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/15.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/16.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/17.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/18.csv,/home/slhuang/Public

Got stdout: b'/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/0.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/1.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/10.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/11.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/12.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/13.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/14.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/15.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/16.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/17.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/18.csv,/home/slhuang/Public

Got stdout: b'/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/0.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/1.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/10.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/11.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/12.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/13.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/14.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/15.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/16.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/17.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/18.csv,/home/slhuang/Public

[{'sample': 'Traceback (most recent call last):\n  File "/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/pandasgen.py", line 318, in generate__dataset\n    dataset.apply_ops(ops)\n  File "/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/pandasgen.py", line 85, in apply_ops\n    raise pd.errors.EmptyDataError\npandas.errors.EmptyDataError\n'}]
['0.csv', '1.csv', '10.csv', '11.csv', '12.csv', '13.csv', '14.csv', '15.csv', '16.csv', '17.csv', '18.csv', '19.csv', '2.csv', '20.csv', '21.csv', '22.csv', '23.csv', '24.csv', '25.csv', '26.csv', '27.csv', '28.csv', '29.csv', '3.csv', '30.csv', '31.csv', '32.csv', '33.csv', '34.csv', '35.csv', '36.csv', '37.csv', '38.csv', '39.csv', '4.csv', '40.csv', '41.csv', '42.csv', '43.csv', '44.csv', '45.csv', '46.csv', '47.csv', '48.csv', '49.csv', '5.csv', '50.csv', '51.csv', '52.csv', '53.csv', '54.csv', '55.csv', '56.csv', '57.csv', '58.csv', '59.csv', '6.csv', '60.csv', '61.csv', '62.csv', '63.csv', '6

...

[{'iloc': 'Traceback (most recent call last):\n  File "/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/pandasgen.py", line 318, in generate__dataset\n    dataset.apply_ops(ops)\n  File "/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/pandasgen.py", line 85, in apply_ops\n    raise pd.errors.EmptyDataError\npandas.errors.EmptyDataError\n'}]
['0.csv', '1.csv', '10.csv', '11.csv', '12.csv', '13.csv', '14.csv', '15.csv', '16.csv', '17.csv', '18.csv', '19.csv', '2.csv', '20.csv', '21.csv', '22.csv', '23.csv', '24.csv', '25.csv', '26.csv', '27.csv', '28.csv', '29.csv', '3.csv', '30.csv', '31.csv', '32.csv', '33.csv', '34.csv', '35.csv', '36.csv', '37.csv', '38.csv', '39.csv', '4.csv', '40.csv', '41.csv', '42.csv', '43.csv', '44.csv', '45.csv', '46.csv', '47.csv', '48.csv', '49.csv', '5.csv', '50.csv', '51.csv', '52.csv', '53.csv', '54.csv', '55.csv', '56.csv', '57.csv', '58.csv', '59.csv', '6.csv', '60.csv', '61.csv', '62.csv', '63.csv', '64.

Got stdout: b'/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/0.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/1.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/10.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/11.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/12.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/13.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/14.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/15.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/16.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/17.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/18.csv,/home/slhuang/Public

Got stdout: b'/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/0.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/1.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/10.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/11.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/12.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/13.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/14.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/15.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/16.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/17.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/18.csv,/home/slhuang/Public

Got stdout: b'/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/0.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/1.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/10.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/11.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/12.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/13.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/14.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/15.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/16.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/17.csv,/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/dataset/18.csv,/home/slhuang/Public

[{'nlargest': 'Traceback (most recent call last):\n  File "/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/pandasgen.py", line 318, in generate__dataset\n    dataset.apply_ops(ops)\n  File "/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/pandasgen.py", line 85, in apply_ops\n    raise pd.errors.EmptyDataError\npandas.errors.EmptyDataError\n'}, {'sample': 'Traceback (most recent call last):\n  File "/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/pandasgen.py", line 318, in generate__dataset\n    dataset.apply_ops(ops)\n  File "/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/pandasgen.py", line 85, in apply_ops\n    raise pd.errors.EmptyDataError\npandas.errors.EmptyDataError\n'}, {'add_rows': 'Traceback (most recent call last):\n  File "/home/slhuang/Public/2019_spring/lineage_inference/code/RELIC/RELIC_Silu/pandasgen.py", line 318, in generate__dataset\n    dataset.apply_ops(ops)\n  Fil

In [28]:
common_edges

{'10,1': [[[9, 9], [9, 9]],
  [[8, 9], [9, 9]],
  [[8, 8], [8, 8]],
  [[6, 5], [6, 5]],
  [[6, 6], [6, 6]],
  [[5, 3], [5, 3]],
  [[9, 7], [9, 7]],
  [[8, 8], [8, 8]],
  [[9, 9], [9, 9]],
  [[9, 8], [9, 8]]],
 '10,2': [[[9, 7], [9, 8]],
  [[9, 7], [9, 7]],
  [[8, 8], [8, 8]],
  [[8, 6], [8, 7]],
  [[9, 6], [9, 7]],
  [[8, 4], [8, 4]],
  [[9, 8], [8, 7]],
  [[9, 9], [9, 9]],
  [[9, 6], [9, 6]],
  [[9, 5], [9, 5]]],
 '10,3': [[[9, 6], [9, 7]],
  [[9, 6], [9, 6]],
  [[9, 9], [9, 9]],
  [[9, 6], [9, 6]],
  [[8, 8], [8, 8]],
  [[8, 7], [8, 7]],
  [[9, 9], [9, 9]],
  [[9, 8], [9, 8]],
  [[9, 7], [9, 7]],
  [[8, 8], [9, 8]]],
 '20,1': [[[14, 13], [14, 13]],
  [[16, 16], [17, 16]],
  [[13, 12], [13, 12]],
  [[10, 10], [10, 10]],
  [[18, 18], [18, 18]],
  [[18, 16], [18, 16]],
  [[12, 12], [12, 12]],
  [[17, 14], [17, 14]],
  [[17, 15], [17, 15]],
  [[16, 15], [16, 15]]],
 '20,2': [[[17, 15], [18, 16]],
  [[19, 19], [19, 19]],
  [[17, 11], [17, 11]],
  [[19, 16], [19, 16]],
  [[19, 17], [19, 17